# Data preparation


## Dependencies


The dependencies used are as follows


In [1]:
from sklearn.preprocessing import LabelEncoder, RobustScaler

from datetime import datetime

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import warnings

warnings.simplefilter("ignore")

## Getting data


Since the latest data collected from kaggle is for the 2023 season, let's check how it works with the first four races of the 2024 season, which are all the races held up to the day on which this work is done.

To do this, you could take the data directly from the official Formula 1 website, which is not what we are going to do. For convenience, we are going to take the data from the same source that is done in kaggle, ergast_api. Note that the data are exactly the same as in the official site, with the advantage that we do not have to make sure that the values of the attributes correspond to the training dataset, since both come from the same source.

Therefore, the next steps will be similar to those performed previously with respect to preprocessing.

_Source: http://ergast.com/mrd/db/#csv_


## Merging data


The first step is to merge the data.


In [2]:
results_df = pd.read_csv("../assets/data/ergast/results.csv")
drivers_df = pd.read_csv("../assets/data/ergast/drivers.csv")
constructors_df = pd.read_csv("../assets/data/ergast/constructors.csv")
races_df = pd.read_csv("../assets/data/ergast/races.csv")
circuits_df = pd.read_csv("../assets/data/ergast/circuits.csv")
status_df = pd.read_csv("../assets/data/ergast/status.csv")
driver_standings_df = pd.read_csv("../assets/data/ergast/driver_standings.csv")
constructor_standings_df = pd.read_csv(
    "../assets/data/ergast/constructor_standings.csv"
)
qualifyings_df = pd.read_csv("../assets/data/ergast/qualifying.csv")

df = pd.merge(results_df, drivers_df, how="left", on="driverId")
df = df.merge(constructors_df, how="left", on="constructorId")
df = df.merge(races_df, how="left", on="raceId")
df = df.merge(circuits_df, how="left", on="circuitId", suffixes=("", "_z"))
df = df.merge(status_df, how="left", on="statusId")
df = df.merge(driver_standings_df, how="left", on=["raceId", "driverId"])
df = df.merge(constructor_standings_df, how="left", on=["raceId", "constructorId"])
df = df.merge(
    qualifyings_df,
    how="left",
    on=["raceId", "driverId", "constructorId"],
    suffixes=("", "_u"),
)

## Renaming data


Subsequently, we rename the columns in the same way as we did in the first section.


In [3]:
df = df.drop(
    [
        "resultId",
        "position_x",
        "positionText_x",
        "time_x",
        "driverId",
        "code",
        "forename",
        "surname",
        "url_x",
        "constructorId",
        "name_x",
        "url_y",
        "raceId",
        "url",
        "fp1_date",
        "fp1_time",
        "fp2_date",
        "fp2_time",
        "fp3_date",
        "fp3_time",
        "quali_date",
        "quali_time",
        "sprint_date",
        "sprint_time",
        "circuitId",
        "name",
        "url_z",
        "statusId",
        "driverStandingsId",
        "points_y",
        "position_y",
        "positionText_y",
        "constructorStandingsId",
        "positionText",
        "qualifyId",
        "number",
        "position_u",
    ],
    axis=1,
)

col_name = {
    "number_x": "carNumber",
    "grid": "positionGrid",
    "positionOrder": "positionFinal",
    "points_x": "pointsDriverEarned",
    "laps": "lapsCompleted",
    "milliseconds": "timeTakenInMillisec",
    "rank": "fastestLapRank",
    "fastestLapSpeed": "maxSpeed",
    "statusId": "status",
    "number_y": "driverNumber",
    "dob": "driverBirth",
    "nationality_x": "driverNationality",
    "nationality_y": "constructorNationality",
    "year": "raceYear",
    "round": "raceRound",
    "name_y": "grandPrix",
    "date": "raceDate",
    "time_y": "raceTime",
    "location": "circuitLocation",
    "country": "circuitCountry",
    "lat": "circuitLat",
    "lng": "circuitLng",
    "alt": "circuitAlt",
    "status": "driverStatus",
    "wins_x": "driverWins",
    "wins_y": "constructorWins",
    "points": "pointsConstructorEarned",
    "position": "constructorPosition",
}

df.rename(columns=col_name, inplace=True)

## Datatype conversion


Now we convert the types.


In [4]:
dates = ["driverBirth", "raceDate"]
for d in dates:
    df[d] = pd.to_datetime(df[d])

h_f, m_f = "%H:%M:%S", "%M:%S.%f"
times = [
    ("fastestLapTime", m_f),
    ("raceTime", h_f),
    ("q1", m_f),
    ("q2", m_f),
    ("q3", m_f),
]
for t, f in times:
    df[t] = pd.to_numeric(
        pd.to_timedelta(
            pd.to_datetime(df[t], format=f, errors="coerce").dt.time.to_numpy(dtype=str)
        ).total_seconds()
        * 1000
    )

numbers = [
    "carNumber",
    "positionGrid",
    "positionFinal",
    "pointsDriverEarned",
    "lapsCompleted",
    "timeTakenInMillisec",
    "fastestLap",
    "fastestLapRank",
    "maxSpeed",
    "driverNumber",
    "raceYear",
    "raceRound",
    "circuitLat",
    "circuitLng",
    "circuitAlt",
    "driverWins",
    "pointsConstructorEarned",
    "constructorPosition",
    "constructorWins",
]
for n in numbers:
    df[n] = pd.to_numeric(df[n], errors="coerce")

df["driverAgeToday"] = np.int64(datetime.today().year - df["driverBirth"].dt.year)
df["driverAgeAtRace"] = np.int64(df["raceDate"].dt.year - df["driverBirth"].dt.year)
df = df.drop(["driverBirth"], axis=1)
df = df.drop(["raceDate"], axis=1)

df = df.replace("\\N", None)

## Race filtering


We filter the races for those that are from the last season.


In [5]:
df = df[df["raceYear"] >= 2024]

We will also check for any nulls.


In [6]:
df.isnull().sum()

carNumber                   0
positionGrid                0
positionFinal               0
pointsDriverEarned          0
lapsCompleted               0
timeTakenInMillisec        39
fastestLap                  3
fastestLapRank              0
fastestLapTime              3
maxSpeed                    3
driverRef                   0
driverNumber                0
driverNationality           0
constructorRef              0
constructorNationality      0
raceYear                    0
raceRound                   0
grandPrix                   0
raceTime                    0
circuitRef                  0
circuitLocation             0
circuitCountry              0
circuitLat                  0
circuitLng                  0
circuitAlt                  0
driverStatus                0
driverWins                  0
pointsConstructorEarned     0
constructorPosition         0
constructorWins             0
q1                          1
q2                         20
q3                         39
driverAgeT

## Missing values: Qualifiers


We will solve the nulls in the same way as we did in the past, so the explanation will be omitted.


In [7]:
first_q_not_null = ~(df["q1"].isnull())
second_q_not_null = ~(df["q2"].isnull())
third_q_not_null = ~(df["q3"].isnull())
mask = first_q_not_null & second_q_not_null & third_q_not_null

In [8]:
mask = first_q_not_null & ~second_q_not_null & ~third_q_not_null
df.loc[mask, ["q2", "q3"]] = df.loc[mask, "q1"]

mask = first_q_not_null & second_q_not_null & ~third_q_not_null
df.loc[mask, "q3"] = df.loc[mask, ["q1", "q2"]].sample(axis=1).squeeze()

In [9]:
qualify_by_race = (
    df[["raceYear", "raceRound", "q1"]]
    .groupby(["raceYear", "raceRound"])
    .agg({"q1": "max"})
)

mask = ~first_q_not_null & ~second_q_not_null & ~third_q_not_null

mg = df.loc[mask]
mg = mg.join(qualify_by_race, how="left", on=["raceYear", "raceRound"], rsuffix="Max")
df.loc[mask, ["q1", "q2", "q3"]] = mg["q1Max"]

## Sorting data


We will sort the data as well as we did before.


In [10]:
df = df.sort_values(by=["raceYear", "raceRound"]).reset_index(drop=True)

## Weather data


First we will add the weather. In this case we had to do web scraping again.


In [11]:
weather_df = pd.read_csv("../assets/data/scraping/weather_2024.csv")
weather_df.rename(columns={"year": "raceYear", "round": "raceRound"}, inplace=True)

df = df.merge(weather_df, how="left", on=["raceYear", "raceRound"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   carNumber                79 non-null     float64
 1   positionGrid             79 non-null     int64  
 2   positionFinal            79 non-null     int64  
 3   pointsDriverEarned       79 non-null     float64
 4   lapsCompleted            79 non-null     int64  
 5   timeTakenInMillisec      40 non-null     float64
 6   fastestLap               76 non-null     float64
 7   fastestLapRank           79 non-null     float64
 8   fastestLapTime           76 non-null     float64
 9   maxSpeed                 76 non-null     float64
 10  driverRef                79 non-null     object 
 11  driverNumber             79 non-null     float64
 12  driverNationality        79 non-null     object 
 13  constructorRef           79 non-null     object 
 14  constructorNationality   79 

We can see that the datatypes are correct so we move on to see if there are any nulls.


In [12]:
mask = df["weather"] == "not found"
proyection = ["raceYear", "raceRound", "grandPrix"]

df.loc[mask, proyection].drop_duplicates()

,raceYear,raceRound,grandPrix


## Circuit data


Secondly, we add the circuit data. In this case, as the circuits are the same without modifications, no scraping has been necessary.


In [13]:
circuits_plus_df = pd.read_csv("../assets/data/scraping/circuits_plus.csv")
circuits_plusplus_df = pd.read_csv("../assets/data/scraping/circuits_plusplus.csv")

df = df.merge(circuits_plus_df, how="left", on=["circuitRef"])
df = df.merge(circuits_plusplus_df, how="left", on=["circuitRef"])

In [14]:
numbers = ["circuitLaps", "circuitDist"]
for n in numbers:
    df[n] = pd.to_numeric(df[n], errors="coerce")

## Additional features


It is followed by the derived attributes.


In [15]:
get_worst_q = lambda row: row[["q1", "q2", "q3"]].min()
get_best_q = lambda row: row[["q1", "q2", "q3"]].max()
get_avg_q = lambda row: row[["q1", "q2", "q3"]].mean()

df["qMin"] = df.apply(get_worst_q, axis=1)
df["qMax"] = df.apply(get_best_q, axis=1)
df["qAvg"] = df.apply(get_avg_q, axis=1)

In [16]:
train_df = pd.read_csv("../assets/data/processed/adding_data.csv")

started_ages = list(
    train_df[["driverRef", "driverAgeStarted"]]
    .set_index(keys="driverRef")
    .to_dict()
    .values()
)[0]

already_exp = list(
    train_df[["driverRef", "driverExpRace"]]
    .set_index(keys="driverRef")
    .to_dict()
    .values()
)[0]

df["driverAgeStarted"] = df.apply(
    lambda x: started_ages.get(x.driverRef, x.driverAgeToday), axis=1
)

exp_by_race = lambda x: [
    sum(df.loc[:i, "driverRef"] == r["driverRef"]) + already_exp.get(r["driverRef"], 0)
    for i, r in x.iterrows()
]

df["driverExpYear"] = df["driverAgeAtRace"] - df["driverAgeStarted"]
df["driverExpRace"] = exp_by_race(df)

In [17]:
podiums = df[df["positionFinal"].isin([1, 2, 3])][
    ["raceYear", "raceRound", "driverRef"]
]
podiums = podiums.groupby(by=["raceYear", "raceRound"]).agg({"driverRef": ",".join})

df = df.merge(
    podiums, how="left", on=["raceYear", "raceRound"], suffixes=("", "Podium")
)
df["driverIsPodium"] = df.apply(
    lambda x: x["driverRef"] in x["driverRefPodium"], axis=1
)
df["driverPodiums"] = df.groupby(["raceYear", "driverRef"])["driverIsPodium"].cumsum()
df = df.drop(["driverRefPodium", "driverIsPodium"], axis=1)

years = df["raceYear"].drop_duplicates().to_numpy()
drivers = df["driverRef"].drop_duplicates().to_numpy()

for year in years:
    for driver in drivers:
        mask = (df["raceYear"] == year) & (df["driverRef"] == driver)
        races = df.loc[mask, "driverPodiums"].iloc[:-1]
        races.loc[-1] = 0
        races.index += 1
        races.sort_index(inplace=True)
        races = races.to_numpy()
        df.loc[mask, "driverPodiums"] = races

In [18]:
for i in range(2, 19):
    position = df[df["positionFinal"].isin([i])][["raceYear", "raceRound", "driverRef"]]
    position = position.groupby(by=["raceYear", "raceRound"]).agg(
        {"driverRef": ",".join}
    )

    df = df.merge(
        position, how="left", on=["raceYear", "raceRound"], suffixes=("", f"Pos{i}")
    )
    df[f"driverIsPos{i}"] = df.apply(
        lambda x: x["driverRef"] in x[f"driverRefPos{i}"], axis=1
    )
    df[f"driverPos{i}"] = df.groupby(["raceYear", "driverRef"])[
        f"driverIsPos{i}"
    ].cumsum()
    df = df.drop([f"driverRefPos{i}", f"driverIsPos{i}"], axis=1)

years = df["raceYear"].drop_duplicates().to_numpy()
drivers = df["driverRef"].drop_duplicates().to_numpy()

features = [
    "driverPos2",
    "driverPos3",
    "driverPos4",
    "driverPos5",
    "driverPos6",
    "driverPos7",
    "driverPos8",
    "driverPos9",
    "driverPos10",
    "driverPos11",
    "driverPos12",
    "driverPos13",
    "driverPos14",
    "driverPos15",
    "driverPos16",
    "driverPos17",
    "driverPos18",
]

for year in years:
    for driver in drivers:
        mask = (df["raceYear"] == year) & (df["driverRef"] == driver)
        races = df.loc[mask, features].iloc[:-1]
        races.loc[-1] = np.zeros(17, dtype=int)
        races.index += 1
        races.sort_index(inplace=True)
        races = races.to_numpy()
        df.loc[mask, features] = races

## Driver ratings data


It is followed by the driver ratings data.


In [19]:
driver_ratings_df = pd.read_csv("../assets/data/scraping/driver_ratings_ea_2024.csv")

In [20]:
mnm = min(driver_ratings_df["driverExp"])
rg = max(driver_ratings_df["driverExp"]) - mnm
arr = (driver_ratings_df["driverExp"] - mnm) / rg
rg2 = df["driverExpRace"].max() - 1
normalized = (arr * rg2) + 1

driver_ratings_df["driverExp"] = round(normalized)

In [21]:
for year, driver, exp, rac, awa, pac, ovr in driver_ratings_df.itertuples(index=False):
    mask = (df["raceYear"] == int(year)) & (
        df["driverRef"].apply(lambda x: x in driver)
    )

    df.loc[mask, "driverExp"] = exp
    df.loc[mask, "driverRac"] = rac
    df.loc[mask, "driverAwa"] = awa
    df.loc[mask, "driverPac"] = pac
    df.loc[mask, "driverOvr"] = ovr

In [22]:
n_features = 5
n_neighbors = 5

driver_exp = dict()

for e in range(1, df["driverExpRace"].max() + 1):
    close_exp = sorted(driver_ratings_df.to_numpy(), key=lambda x: np.abs(x[2] - e))[
        :n_neighbors
    ]
    avg_ratings = []
    for i in range(2, n_features + 2):
        ratings = []
        for j in range(n_neighbors):
            ratings.append(close_exp[j][i])
        avg_ratings.append(round(sum(ratings) / n_neighbors))
    driver_exp[e] = avg_ratings

m = df["driverOvr"].isnull()
p = "driverExpRace"
df["driverExp"] = df["driverExp"].fillna(df.loc[m, p].apply(lambda x: driver_exp[x][0]))
df["driverRac"] = df["driverRac"].fillna(df.loc[m, p].apply(lambda x: driver_exp[x][1]))
df["driverAwa"] = df["driverAwa"].fillna(df.loc[m, p].apply(lambda x: driver_exp[x][2]))
df["driverPac"] = df["driverPac"].fillna(df.loc[m, p].apply(lambda x: driver_exp[x][3]))
df["driverOvr"] = df["driverOvr"].fillna(df.loc[m, p].apply(lambda x: driver_exp[x][4]))

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 75 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   carNumber                79 non-null     float64
 1   positionGrid             79 non-null     int64  
 2   positionFinal            79 non-null     int64  
 3   pointsDriverEarned       79 non-null     float64
 4   lapsCompleted            79 non-null     int64  
 5   timeTakenInMillisec      40 non-null     float64
 6   fastestLap               76 non-null     float64
 7   fastestLapRank           79 non-null     float64
 8   fastestLapTime           76 non-null     float64
 9   maxSpeed                 76 non-null     float64
 10  driverRef                79 non-null     object 
 11  driverNumber             79 non-null     float64
 12  driverNationality        79 non-null     object 
 13  constructorRef           79 non-null     object 
 14  constructorNationality   79 

Now we will check if there are any nulls.


In [24]:
df.isnull().sum()

carNumber                   0
positionGrid                0
positionFinal               0
pointsDriverEarned          0
lapsCompleted               0
timeTakenInMillisec        39
fastestLap                  3
fastestLapRank              0
fastestLapTime              3
maxSpeed                    3
driverRef                   0
driverNumber                0
driverNationality           0
constructorRef              0
constructorNationality      0
raceYear                    0
raceRound                   0
grandPrix                   0
raceTime                    0
circuitRef                  0
circuitLocation             0
circuitCountry              0
circuitLat                  0
circuitLng                  0
circuitAlt                  0
driverStatus                0
driverWins                  0
pointsConstructorEarned     0
constructorPosition         0
constructorWins             0
q1                          0
q2                          0
q3                          0
driverAgeT

## Encoding and normalization


Now we proceed to encode and normalize.


In [25]:
X = df.drop(
    [
        "positionFinal",
        "pointsDriverEarned",
        "lapsCompleted",
        "timeTakenInMillisec",
        "fastestLap",
        "fastestLapRank",
        "fastestLapTime",
        "maxSpeed",
        "driverStatus",
        "pointsConstructorEarned",
        "constructorPosition",
    ],
    axis=1,
)

enc = LabelEncoder()
for c in X.columns:
    if X[c].dtype == "object":
        X[c] = enc.fit_transform(X[c])

scaler = RobustScaler()
X = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns)

Finally, we write down the results for next section


In [26]:
df.to_csv("../assets/data/processed/testing.csv", index=False)
X.to_csv("../assets/data/processed/testing_X.csv", index=False)